In [1]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility as iv
from math import exp, sqrt, log, fabs
import matplotlib.pyplot as plt
from itertools import repeat
import os
import calendar 
import glob

In [2]:
work_dir = os.getcwd()
work_dir
Option_path = os.path.join(work_dir, 'Data', 'Option')
Future_path = os.path.join(work_dir, 'Data', 'Futures')
Rate_path= os.path.join(work_dir, 'Data', 'Rate')

In [3]:
Option_path

'C:\\Users\\Evan\\Documents\\GitHub\\TXO_TX_Strategy\\Data\\Option'

## Calculate the Date Difference

In [4]:
def dates_delta(start_time, maturity):
    'Compute the distance between two TRADE_DATEs'
    s = start_time
    
    d0 = date(year = int(s/10000), month = int((s % 10000)/100), day = int(s % 100))
    d1 = get_expireDate(int(maturity/100), int(maturity % 100))

    t = np.busday_count(d0, d1)/252
    if t < 0:
        print("The input dates error!")
    return t

In [24]:
def trans_dt(d):
    d_new = date(year = int(d/10000), month = int((d % 10000)/100), day = int(d % 100))
    return d_new

In [5]:
def get_expireDate(year, month):
    c = calendar.Calendar(firstweekday=calendar.SATURDAY)
    monthcal = c.monthdatescalendar(year, month)
    monthly_expire_date = monthcal[2][-3]
    return monthly_expire_date

In [6]:
def get_Future_List(): 
    extension = 'csv'
    os.chdir(Future_path)
    Fut_csv_List = glob.glob('*.{}'.format(extension))
    return Fut_csv_List

In [7]:
def get_Option_List(): 
    extension = 'csv'
    os.chdir(Option_path)
    Opt_csv_List = glob.glob('*.{}'.format(extension))
    return Opt_csv_List

In [63]:
def get_zcb_rate(transDate):
    if ~isinstance(transDate, date): 
        transDate = np.datetime64(trans_dt(transDate))
    
    rate =  pd.read_csv(os.path.join(Rate_path, 'ZCB.csv'))
    rate['Date'] = pd.to_datetime(rate['Date'])
    zcb = float(rate[rate.Date == transDate]['ZCB'])
    
    return zcb

In [8]:
if __name__ ==  '__main__':
    Opt_csv_List = get_Option_List()
    Option =  pd.read_csv(os.path.join(Option_path, Opt_csv_List[0]), encoding = 'cp950')
    

In [9]:
Option.head()

,TransDate,Product,ExcercisePrice,Maturity,Right,TransTime,TransPrice,Volumn
0,20200313,TXO,8200.0,202003.0,P,84500.0,19.5,1.0
1,20200313,TXO,8200.0,202003.0,P,84500.0,19.5,5.0
2,20200313,TXO,8200.0,202003.0,P,84500.0,19.5,13.0
3,20200313,TXO,8200.0,202003.0,P,84500.0,19.5,5.0
4,20200313,TXO,8200.0,202003.0,P,84500.0,19.5,1.0


In [20]:
start_time = Option.TransDate[0]
maturity = Option.Maturity[0]

In [11]:
rate = pd.read_csv(os.path.join(Rate_path, 'ZCB.csv'))